In [ ]:
import numpy as np
from matplotlib import pyplot as plt

Define the problem parameters


In [ ]:
A = np.array([[1, 3], [3, 1]])  # low dimensions to plot it, you can test larger sizes
b = np.array([-1, 2])

lmbda = 0.5

The function to be minimized is $\frac{1}{2}\|Ax-b\|_2^2 + \lambda \|x\|_1$


In [ ]:
f = lambda x: 0.5 * np.sum((A @ x - b) ** 2)
F = lambda x: f(x) + lmbda * np.sum(np.abs(x))

# derivative of f from matrix calculus
df = lambda x: A.T @ (A @ x) - A.T @ b

Plot the function


In [ ]:
# this function has been prepared only for the visualization sake, no need to go through this but it renders some nice
# graphics :)
Fplot = (
    lambda r1, r2: (r1 * A[0, 0] + r2 * A[0, 1] - b[0]) ** 2
    + (r1 * A[1, 0] + r2 * A[1, 1] - b[1]) ** 2
    + lmbda * (np.abs(r1) + np.abs(r2))
)
xx, yy = np.meshgrid(np.arange(-10, 10, 1), np.arange(-10, 10, 1))

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.plot_surface(
    xx, yy, Fplot(xx, yy), edgecolor=[0, 0, 1], alpha=0.5, facecolor=[0, 1, 1]
)

## ISTA with backtracking

Parameters of ISTA


In [ ]:
MAX_ITER = 1e3
TOL_DIST_X = 1e-6
# initial value for alpha
alpha = 1
# parameter for backtracking
eta = 0.99

Initialization: test different inizializations, the function is convex, you always converge to the same solution


In [ ]:
x = np.array([-5, 10])

grad_norm = 1e10
distanceX = 1e10

# initialize the list with all the estimates
all_x = [x]
x_past = x

Main loop


In [ ]:
cnt = 0
alpha_current = alpha
while cnt < MAX_ITER and distanceX > TOL_DIST_X:
    # store previous x for distance calculation
    x_prev = x.copy()

    # compute the argument of the proximal operator
    x_current = x - alpha_current * df(x)

    # perform soft thresholding of x
    x_current = np.sign(x_current) * np.maximum(
        np.abs(x_current) - alpha_current * lmbda, 0
    )

    # iterate the backtracking procedure until the stopping criterion is met
    while (
        f(x_current)
        > f(x)
        + df(x).T @ (x_current - x)
        + (1 / (2 * alpha_current)) * np.linalg.norm(x_current - x) ** 2
    ):
        cnt = cnt + 1
        # update alpha
        alpha_current = eta * alpha_current

        # compute again x
        x_current = x - alpha_current * df(x)

        # perform soft thresholding
        x_current = np.sign(x_current) * np.maximum(
            np.abs(x_current) - alpha_current * lmbda, 0
        )

    # update x
    x = x_current.copy()

    # compute the stopping criteria
    distanceX = np.linalg.norm(x - x_prev)

    # store the estimate
    all_x.append(x.copy())

    cnt += 1

Plot all the estimates


In [ ]:
# plot the new estimate
xxplot = [x[0] for x in all_x]
yyplot = [x[1] for x in all_x]
zzplot = Fplot(np.array(xxplot), np.array(yyplot))

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection="3d")
ax.plot_surface(
    xx, yy, Fplot(xx, yy), edgecolor=[0, 0, 1], alpha=0.5, facecolor=[0, 1, 1]
)
ax.plot3D(xxplot, yyplot, zzplot, "r-o")

Compare the number of iteration needed for convergence with backtracking with the number of iteration required using the optimal alpha


In [ ]:
print(
    f"nr of iteration of ISTA with backtracking (before stopping criteria met): {cnt}\n"
)
print(f"Solution of ISTA: [{x_current[0]:.4f}, {x_current[1]:.4f}]\n")
print(f"Value of the functional: {F(x_current):.4f}\n")

## FISTA

Parameters of FISTA


In [ ]:
MAX_ITER = 1e3
TOL_DIST_X = 1e-6
# set the optimal value for alpha
alpha = 1 / np.max(np.linalg.eigvals(A.T @ A))

Initialization: test different inizializations, the function is convex, you always converge to the same solution


In [ ]:
# initialize x
x = np.array([-5, 10])

# initialize y
y = x.copy()

# initialize gamma
gamma = 1

grad_norm = 1e10
distanceX = 1e10

# initialize the list with all the estimates
all_x = [x]

Main loop


In [ ]:
cnt = 0
while cnt < MAX_ITER and distanceX > TOL_DIST_X:
    # store previous x for distance calculation
    x_prev = x.copy()

    # compute the argument of the proximal operator
    x_current = y - alpha * df(y)

    # perform soft thresholding of x
    x_current = np.sign(x_current) * np.maximum(np.abs(x_current) - alpha * lmbda, 0)

    # update gamma
    gamma_new = (1 + np.sqrt(1 + 4 * gamma**2)) / 2

    # update y
    y = x_current + ((gamma - 1) / gamma_new) * (x_current - x)

    # update x and gamma for next iteration
    x = x_current.copy()
    gamma = gamma_new

    # compute the stopping criteria
    distanceX = np.linalg.norm(x - x_prev)

    # store the estimate
    all_x.append(x.copy())

    cnt += 1

Plot all the estimates


In [ ]:
# plot the new estimate
xxplot = [x[0] for x in all_x]
yyplot = [x[1] for x in all_x]
zzplot = Fplot(np.array(xxplot), np.array(yyplot))

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection="3d")
ax.plot_surface(
    xx, yy, Fplot(xx, yy), edgecolor=[0, 0, 1], alpha=0.5, facecolor=[0, 1, 1]
)
ax.plot3D(xxplot, yyplot, zzplot, "r-o")

In [ ]:
print(f"nr of iteration of FISTA (before stopping criteria met): {cnt}\n")
print(f"Solution of FISTA: [{x_current[0]:.4f}, {x_current[1]:.4f}]\n")
print(f"Value of the functional: {F(x_current):.4f}\n")